## Generating Training Sest for Stage 4

Fixed: Depth Increments (0.01 m)

Variables (Inputs) : CPTu Dataset, RBC

Output : Optimised Volume (Minimum Volume for said Capacity)

In [7]:
import torch
from Functions_CPTu_Cap_Copy import *
from itertools import product

import torch

# Set the device to GPU if CUDA is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


## Importing CPTu Datasets

In [2]:
import pandas as pd
import numpy as np
## 
# 
# CPTu_data = pd.read_csv('CPTu_train_set.csv', header=None, skiprows=1)
CPTu_data = pd.read_csv('CPTu_val_set.csv', header=None, skiprows=1)
# CPTu_data = pd.read_csv('CPTu_test_set.csv', header=None, skiprows=1)

print(np.shape(CPTu_data))
print(type(CPTu_data))
print(CPTu_data.shape[1] // 4)

(3081, 12)
<class 'pandas.core.frame.DataFrame'>
3


## Creating Length, Diameter and RBC Inputs

In [3]:
l = np.arange(0.1, 30.81, 0.1)

RBC = np.arange(50, 2200, 50) # 10 values - when interval 200
b = np.arange(0.4, 1.6, 0.1) # 7 values


print(np.shape(RBC))
# zip = list(product(RBC, b))
# print((zip[:50]))
# print(zip[10][1])
# print(len(zip))

# print(zip[1][0])

(43,)


## Implementing Training Loop

In [8]:
print(np.shape(CPTu_data), np.shape(l), np.shape(b), np.shape(RBC))

print(type(CPTu_data))

CPTu_data_np = CPTu_data.to_numpy()
print(type(CPTu_data_np))

# Assuming CPTu_data, l, b, RBC are already loaded as NumPy arrays or similar
CPTu_data_tensor = torch.tensor(CPTu_data_np, dtype=torch.float32).to(device)
l_tensor = torch.tensor(l, dtype=torch.float32).to(device)
b_tensor = torch.tensor(b, dtype=torch.float32).to(device)
RBC_tensor = torch.tensor(RBC, dtype=torch.float32).to(device)


outputs = []
for i in range(len(RBC)):
    print(i)
    
    temp_outputs = []

    temp_RBC = RBC[i]

    print(temp_RBC)
    temp_outputs.append(s4_spec_bear_cap(CPTu_data_np, l, b, temp_RBC))

    outputs.append(temp_outputs)

print(np.shape(outputs))
# print(outputs)

# print(spec_bear_cap(CPTu_data_np, l, b, 200))
# 
# print(np.shape(spec_bear_cap(CPTu_data_np, l, b, 200)))

(3081, 12) (308,) (13,) (43,)
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
0
50
1
100
2
150
3
200
4
250
5
300
6
350
7
400
8
450
9
500
10
550
11
600
12
650
13
700
14
750
15
800
16
850
17
900
18
950
19
1000
20
1050
21
1100
22
1150
23
1200
24
1250
25
1300
26
1350
27
1400
28
1450
29
1500
30
1550
31
1600
32
1650
33
1700
34
1750
35
1800
36
1850
37
1900
38
1950
39
2000
40
2050
41
2100
42
2150
(43, 1, 3, 1, 4)


## Changing Data Structure

In [9]:
outputs = [output.cpu().numpy() for output in outputs]  # Assuming outputs is a list of tensors
print(np.shape(outputs))

AttributeError: 'list' object has no attribute 'cpu'

In [10]:
outputs_squeezed = np.squeeze(outputs, axis=1)

print(np.shape(outputs_squeezed))
print(outputs_squeezed[3])

(43, 3, 1, 4)
[[[200.           0.3          0.4        213.50472843]]

 [[200.           1.2          0.4        205.04285551]]

 [[200.           1.8          0.4        211.75526405]]]


## Altering CPTu Structure

In [11]:
columns_to_drop = [4 * i + j for i in range((CPTu_data.shape[1] // 4)) for j in range(2)]

# Drop the columns
CPTu_data_dropped = CPTu_data.drop(CPTu_data.columns[columns_to_drop], axis=1)

# Flattening the remaining columns for each borehole group
flattened_CPTu_groups = []
for i in range(0, CPTu_data_dropped.shape[1], 2):  # Iterate over each pair of remaining columns
    group = CPTu_data_dropped.iloc[:, i:i+2].values
    flattened_group = np.concatenate(group.T)  # Flatten the group transposed to concatenate by columns
    flattened_CPTu_groups.append(flattened_group)

# Output the shape and type for confirmation
print(np.shape(flattened_CPTu_groups))  # Expected shape: (no_BH, 20) if original was 10 samples per column
print(type(flattened_CPTu_groups))
print(flattened_CPTu_groups[0])

(3, 6162)
<class 'list'>
[0.   0.21 0.4  ... 0.   0.   0.  ]


## Export to CSV

In [12]:
# Setup the columns for DataFrame
columns = ['RBC'] + [f'CPTu_{i}' for i in range(1, 6163)] + ['Optim_L'] + ['Optim_b'] + ['Actual_Cap']

# Calculate the total number of rows in the final DataFrame
total_rows = len(RBC) * (CPTu_data.shape[1] //4)

# Pre-allocate a numpy array for all data
data_array = np.empty((total_rows, len(columns)))

print(CPTu_data.shape[1] //4)

# Fill the data array
current_row = 0
for i in range(len(RBC)):
    print(i)
    for j in range(CPTu_data.shape[1] // 4):
        # Each row is Length + Flattened CPTU data + Output
        data_array[current_row, 0] = outputs_squeezed[i][j][0][0]
        data_array[current_row, 1:6163] = flattened_CPTu_groups[j]
        data_array[current_row, 6163] = outputs_squeezed[i][j][0][1] 
        data_array[current_row, 6164] = outputs_squeezed[i][j][0][2]
        data_array[current_row, 6165] = outputs_squeezed[i][j][0][3]
        current_row += 1

# Create a DataFrame from the numpy array
df = pd.DataFrame(data_array, columns=columns)

# Save the DataFrame to a CSV file
df.to_csv('Stage_4_vald_merged_reduced.csv', index=False)
print("CSV file has been created with the shape:", df.shape)


3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
CSV file has been created with the shape: (129, 6166)


In [ ]:
### Check CSV files as they seem to appended with a lot of zeros at the bottom